In [1]:

%run ../../load_magic/storage.py
%run ../../load_magic/paths.py
%run ../../load_magic/lists.py
%run ../../load_magic/environment.py
%run ../../load_magic/dataframes.py

from bs4 import BeautifulSoup as bs
import networkx as nx
import re
import urllib.request

notebook_path = get_notebook_path()
print(notebook_path)
s = Storage()
print([fn for fn in dir(s) if not fn.startswith('_')])

%pprint
%who

C:\Users\dev\Documents\repositories\notebooks\Miscellaneous\ipynb\Untitled.ipynb
data_folder: ../data/
saves_folder: ../saves/
['attempt_to_pickle', 'data_csv_folder', 'data_folder', 'encoding_type', 'load_csv', 'load_dataframes', 'load_object', 'save_dataframes', 'saves_csv_folder', 'saves_folder', 'saves_pickle_folder', 'store_objects']
Pretty printing has been turned OFF
SequenceMatcher	 Storage	 bs	 check_4_doubles	 check_for_typos	 conjunctify_nouns	 copy_all_notebooks_from_the_repositories	 copyfile	 csv	 
encoding	 example_iterrows	 get_column_descriptions	 get_git_lfs_track_commands	 get_max_rsquared_adj	 get_module_version	 get_notebook_path	 get_page_tables	 get_specific_gitignore_files	 
humanize_bytes	 ipykernel	 json	 math	 notebook_path	 notebookapp	 nx	 os	 pd	 
pickle	 print_all_files_ending_starting_with	 print_all_files_ending_with	 print_all_files_starting_with	 re	 remove_empty_folders	 s	 similar	 sm	 
subprocess	 sys	 time	 urllib	 where_are_any_wav_files	 


In [2]:

def get_page_soup(page_url):
    with urllib.request.urlopen(page_url) as response:
        page_html = response.read()
    page_soup = bs(page_html, 'html.parser')
    
    return page_soup

In [5]:

tables_url = 'https://factvsfitness.com/histamine-intolerance-food-list/'
hystamine_path = os.path.join(s.data_folder, 'html', 'low_hystamine_diet.html')
#page_soup = get_page_soup(tables_url)
tables_list = get_page_tables(hystamine_path)

[(0, (13, 5))]


In [6]:

#itertools.combinations(iterable, r)
with open(hystamine_path, 'r', encoding=s.encoding_type) as f:
    html_str = f.read()
strong_li_list = re.split('<[^>]+data-th="([^"]+)">', html_str, 0)
strong_li_list = [re.sub(r'>\s+', '>', re.sub(r'\s+<', '<', s)) for s in strong_li_list[1:]]
n = len(strong_li_list)

In [7]:

rows_list = []
row_dict = {}
for i in range(0, n, 2):
    strong_li_str = strong_li_list[i+1]
    allowed_restricted = strong_li_list[i]
    if allowed_restricted == 'ALLOWED':
        row_dict['is_restricted'] = False
    elif allowed_restricted == 'RESTRICTED':
        row_dict['is_restricted'] = True
    for match_obj in re.finditer(r'<(strong|li)>([^<]+)</\1>', strong_li_str):
        #print(match_obj.group(1), match_obj.group(2))
        if match_obj.group(1) == 'strong':
            row_dict['food_group'] = match_obj.group(2)
        elif match_obj.group(1) == 'li':
            row_dict['food_name'] = match_obj.group(2)
            #print(row_dict)
            rows_list.append(row_dict.copy())
    #print(allowed_restricted, strong_li_str)
hystamine_df = pd.DataFrame(rows_list)
columns_list = ['is_restricted']
#[fn for fn in dir(hystamine_df) if 'dup' in fn.lower()]
s.store_objects(hystamine_df=hystamine_df)
hystamine_df.drop_duplicates(subset=columns_list)

Pickling to C:\Users\dev\Documents\repositories\notebooks\Miscellaneous\saves\pickle\hystamine_df.pickle


,food_group,is_restricted,food_name
0,Vegetables,False,All fresh vegetables except those listed opposite
1,Vegetables,True,Eggplant


In [8]:

mask_series = hystamine_df['is_restricted']
restricted_list = hystamine_df[mask_series]['food_name'].tolist()
unrestricted_list = hystamine_df[~mask_series]['food_name'].tolist()

In [9]:

restricted_list

['Eggplant', 'Pumpkin', 'Sauerkraut', 'Spinach', 'Tomato', 'Avocado', 'Olives', 'Pickled', 'vegetables', 'Citrus fruits', 'Banana', 'Cherry', 'Cranberry', 'Currant', 'Date', 'Loganberry', 'Nectarine', 'Orange', 'Papaya', 'Peach', 'Pineapple', 'Prunes', 'Plums', 'Raisins', 'Raspberries', 'Strawberries', 'Fruit dishes,jams, juices made with restricted ingredients', 'All fish (unless fresh caught)', 'All shellfish', 'Leftover/refrigerated meats', 'Processed, cured or smoked meats', 'Raw egg whites (as in some eggnog, hollandaise sauce, milk shakes)', 'All fats and oils with colours or preservatives', 'Prepared gravy', 'Commercial salad dressings', 'Hydrolyzed lecithin', 'Anise', 'Cinnamon', 'Cloves', 'Curry powder', 'Paprika/cayenne', 'Nutmeg', 'Seasoning packets with restricted ingredients', 'Foods labeled “with spices”', 'Walnuts', 'Cashews', 'Soy', 'Lentils', 'Beans', 'Peanuts', 'Foods cooked in oils containing hydrolyzed lecithin, BHA or BHT Baking Mixes', 'Dry dessert mixes', 'All ch

In [10]:

unrestricted_list

['All fresh vegetables except those listed opposite', 'Apple', 'Cantaloupe', 'Figs', 'Grapefruit', 'Grapes', 'Honeydew', 'Mango', 'Pear', 'Rhubarb', 'Lychee', 'Persimmon', 'Blackberries', 'Blueberries', 'Watermelon', 'Coconut', 'All fresh meat and poultry. Must be bought and cooked fresh.', 'Plain eggs (ensure whites are fully cooked)', 'Pure butter', 'All vegetable oils and oils of allowed', 'foods which contain no additives', 'Coconut oil', 'Meat drippings and fat', 'Homemade gravy', 'Homemade salad dressings with allowed ingredients', 'All fresh herbs and spices except those listed opposite', 'All nuts should be eliminated for the first two weeks on the diet. After this period, nuts aside from those listed opposite may be reintroduced individually to examine tolerance. Begin with macadamias and chestnuts.', 'All legumes should be eliminated for the first two weeks on the diet. After this period, legumes aside from those listed opposite may be reintroduced individually to examine tol

In [11]:

tables_list[0]

,FOOD GROUP,ALLOWED,ALLOWED.1,RESTRICTED,RESTRICTED.1
0,Vegetables,All fresh vegetables except those listed opposite,All fresh vegetables except those listed opposite,Eggplant Pumpkin Sauerkraut Spinach Tomato,Avocado Olives Pickled vegetables
1,Fruits,Apple Cantaloupe Figs Grapefruit Grapes H...,Rhubarb Lychee Persimmon Blackberries Blue...,Citrus fruits Banana Cherry Cranberry Curr...,Pineapple Prunes Plums Raisins Raspberries...
2,"Meat, Poultry and Fish",All fresh meat and poultry. Must be bought and...,All fresh meat and poultry. Must be bought and...,All fish (unless fresh caught) All shellfish ...,All fish (unless fresh caught) All shellfish ...
3,Eggs,Plain eggs (ensure whites are fully cooked),Plain eggs (ensure whites are fully cooked),"Raw egg whites (as in some eggnog, hollandaise...","Raw egg whites (as in some eggnog, hollandaise..."
4,Fats and Oils,Pure butter All vegetable oils and oils of al...,Pure butter All vegetable oils and oils of al...,All fats and oils with colours or preservative...,All fats and oils with colours or preservative...
5,Spices and Herbs,All fresh herbs and spices except those listed...,All fresh herbs and spices except those listed...,Anise Cinnamon Cloves Curry powder Paprika...,Anise Cinnamon Cloves Curry powder Paprika...
6,Nuts and seeds,All nuts should be eliminated for the first tw...,All nuts should be eliminated for the first tw...,Walnuts Cashews,Walnuts Cashews
7,Legumes*,All legumes should be eliminated for the first...,All legumes should be eliminated for the first...,Soy Lentils Beans Peanuts,Soy Lentils Beans Peanuts
8,Breads and Cereals*,Total of Â½ cup per day maximum of the followi...,Total of Â½ cup per day maximum of the followi...,Foods cooked in oils containing hydrolyzed lec...,Foods cooked in oils containing hydrolyzed lec...
9,Milk and Dairy*,"After two weeks on this diet, plain milk can b...","After two weeks on this diet, plain milk can b...",All cheese All yogurt (unless specifically cu...,All cheese All yogurt (unless specifically cu...


In [25]:

#s.load_csv(csv_name='btvc_legalillegal_list')
csv_path = os.path.join(s.data_csv_folder, 'btvc_legalillegal_list.csv')
btvc_legalillegal_df = pd.read_csv(csv_path, encoding=s.encoding_type, delimiter='|')
s.store_objects(btvc_legalillegal_df=btvc_legalillegal_df)
btvc_legalillegal_df

Pickling to C:\Users\dev\Documents\repositories\notebooks\Miscellaneous\saves\pickle\btvc_legalillegal_df.pickle


,item_name,legal_illegal,additional_information
0,Acorn Squash,Legal,NaN
1,Agar-agar,Illegal,Contains polysaccharides. See wikipedia:agaragar
2,Agave syrup,Illegal,NaN
3,Algae,Illegal,We do not use algae (Spirulina) because IBD in...
4,Allspice,Legal,As long as you're just using Allspice as a coo...
...,...,...,...
429,Yams,Illegal,NaN
430,Yoghurt (commercial),Illegal,Eating commercial yoghurt is not permitted. Co...
431,Yoghurt (homemade),Legal,"See, yoghurt in the Knowledge Base section for..."
432,Yucca Root,Illegal,NaN


In [27]:

mask_series = (btvc_legalillegal_df.legal_illegal == 'Legal')
btvc_legalillegal_df[mask_series]['item_name'].tolist()

['Acorn Squash', 'Allspice', 'Almond butter', 'Almond milk', 'Almond oil', 'Almonds', 'Anchovies', 'Apple Cider', 'Apples', 'Apricots', 'Artichokes (French)', 'Ascorbic acid', 'Asiago cheese', 'Asparagus', 'Aspartame', 'Aspartic acid', 'Avocado oil', 'Avocadoes', 'Bacon', 'Baking soda', 'Bananas', 'Basil', 'Bay Leaf', 'Beef', 'Beets', 'Berries', 'Black beans', 'Black radish', 'Blue cheese', 'Bok Choy', 'Bourbon', 'Brazil nuts', 'Brick cheese', 'Brie cheese', 'Broccoli', 'Brussell sprouts', 'Butter', 'Butternut squash', 'Cabbage', 'Camembert cheese', 'Canned fish', 'Canola oil', 'Cantaloupe', 'Capers', 'Carrots', 'Cashews', 'Cauliflower', 'Celeriac /celery root', 'Celery', 'Cellulose (in supplements)', 'Chard', 'Cheddar cheese', 'Cheese', 'Cherimoya', 'Cherries', 'Chestnuts', 'Cilantro', 'Cinnamon', 'Citric Acid', 'Club soda', 'Coconut', 'Coconut milk', 'Coconut oil', 'Coffee', 'Colby cheese', 'Collard greens', 'Corn oil', 'Courgette', 'Cranberry juice', 'Cucumbers', 'Custard apple', 'C